In [5]:
#import libraries and autotime
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.58 ms


In [6]:
#create urls for each term/year page
years = range(1990,2018)
urls = []
for year in years:
    url = 'https://www.oyez.org/cases/' + str(year)
    urls.append(url)


time: 1.89 ms


In [7]:
#get html for each term/year page
htmls = []
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
for url in urls:    
    driver.get(url)
    sleep(1)
    html = driver.page_source
    htmls.append(html)

time: 42.6 s


In [8]:
#close webdriver
driver.close()

time: 116 ms


In [9]:
#beautiful soup the html
soups = []
for html in htmls:
    soup = BeautifulSoup(html, 'lxml')
    soups.append(soup)

time: 3.31 s


In [10]:
#extract the urls for the court cases on each page
case_links = []
for soup in soups:
    for i in soup.find_all('a', href=True):
        if len(i['href'])>10 and 'cases/' in i['href']:            
            case_links.append('https://www.oyez.org/' + i['href'])

time: 1.13 s


In [11]:
#cleaning
case_series = pd.Series(case_links)
case_series.drop_duplicates(inplace=True)
case_series = case_series[4:]

time: 4.2 ms


In [12]:
#navigate to each page, save html, find oral argument link, follow link, and save html
#long sleep times due to slow site.  tried using shorter times but data didn't always load
htmls1 = []
htmls2 = []
links = []
list_test = []
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
for url in case_series:    
    #go to link
    driver.get(url)
    #wait b/c javascript
    sleep(5)
    #get html
    html1 = driver.page_source
    htmls1.append(html1)
    #beautiful soup
    soup = BeautifulSoup(html1, 'lxml')
    for i in soup.find_all('a', {'class':'ng-binding ng-isolate-scope'}):
        if 'argument' in i['iframe-url']:
            if 'https://apps.oyez.org' in i['iframe-url']:
                driver.get(str(i['iframe-url']))
                links.append(str(i['iframe-url']))
                sleep(10)
                html2 = driver.page_source
                htmls2.append(html2)
                list_test.append([str(i['iframe-url']),html2,html1,url])
                

time: 9h 56min 24s


In [13]:
driver.close()

time: 107 ms


In [20]:
list_text2 = []
for i in list_test:
    soup1 = BeautifulSoup(i[2], 'lxml')
    soup2 = BeautifulSoup(i[1], 'lxml')
    list_text2.append([i[0],soup1,soup2,i[3]])

time: 10min 59s


In [44]:
#extract necessary information.  
list_text3 = []
for term in list_text2:
    audio_link = []
    docket = []
    petitioner = []
    respondent = []
    decided = []
    decision = []
    transcript = []
    pet_law = []
    resp_law = []
    for i in term[2].find_all('section',{'ng-repeat':'turn in section.turns'}):
        verb = ' '
        speaker= []
        for x in i.find('h4'):
            speaker = x
        for y in i.find_all('p', {'ng-repeat':'textBlock in turn.text_blocks'}):
            verb = verb + y.text
        if len(speaker) == 0:
            speaker = np.NaN
        transcript.append([speaker,verb])
    for i in term[1].find_all('div',{'class':'cell ng-binding'}):
        if 'Docket' in i.text:
            docket = i.text
        if 'Petitioner' in i.text:
            petitioner = i.text
        if 'Respondent' in i.text:
            respondent = i.text
    for g in term[1].find_all('div',{'ng-repeat':'advocate in case.advocates'}):
        if 'Petitioner' in g.text:
            pet_law = g.text
        if 'Respondents' in g.text:
            resp_law = g.text
        
    for x in term[1].find_all('a', {'class':'ng-binding ng-isolate-scope'}):  
        
        if 'oral_argument_audio' in str(x['iframe-url']):
            audio_link.append(str(x['iframe-url']))
    for y in term[1].find_all('div', {'ng-repeat':'chron in case.timeline'}):
        if 'Decided' in y.text:
            decided = y.text[9:]
    for f in term[1].find_all('span',{'ng-if':'decision.winning_party'}):
        decision = f.text
        
    list_text3.append([term[0], audio_link, docket[11:], respondent[11:], petitioner[11:] ,pet_law, resp_law, decided, decision[5:],transcript,term[3]]) 

time: 3min 35s


In [45]:
#create dictionary of speakers and the number of times they spoke and the words used by each speaker
#adding information that was gathered in the last step
dict_list_speakers = []
dict_list_text = []
list_text4 = []
for i in list_text3:
    temp_dict = {}
    temp_dict2 = {}
    for x in i[9]:
        if x[0] in temp_dict:
            temp_dict[x[0]] = temp_dict.get(x[0], 0) + 1
        else:
            temp_dict.update({x[0]:1})
    for x in i[9]:
        if x[0] in temp_dict:
            temp_dict2[x[0]] = str(temp_dict2.get(x[0], 0)) + x[1].encode('utf-8').strip()
        else:
            temp_dict2.update({x[0]:x[1]})
    list_text4.append([i[0],i[2],i[3], i[4],i[5],i[6],i[7],i[8], temp_dict, temp_dict2, i[10]])
    dict_list_speakers.append(temp_dict)
    dict_list_text.append(temp_dict2)

time: 3.29 s


In [40]:
dict_list_speakers[1000]

{nan: 97,
 u'Anthony M. Kennedy': 13,
 u'Antonin Scalia': 36,
 u'David H. Souter': 14,
 u'James W. Ellis': 63,
 u'John Paul Stevens': 11,
 u'Pamela A. Rumpz': 60,
 u'Ruth Bader Ginsburg': 10,
 u"Sandra Day O'Connor": 22,
 u'Stephen G. Breyer': 9,
 u'The Marshal': 1,
 u'William H. Rehnquist': 17}

time: 8.13 ms


In [41]:
list_text4[1000]

['https://apps.oyez.org/player/#/rehnquist10/oral_argument_audio/23138',
 u' 00-8452 ',
 u' Virginia ',
 u' Atkins ',
 [],
 [],
 u'Jun 20, 2002 ',
 u'Atkins',
 {nan: 97,
  u'Anthony M. Kennedy': 13,
  u'Antonin Scalia': 36,
  u'David H. Souter': 14,
  u'James W. Ellis': 63,
  u'John Paul Stevens': 11,
  u'Pamela A. Rumpz': 60,
  u'Ruth Bader Ginsburg': 10,
  u"Sandra Day O'Connor": 22,
  u'Stephen G. Breyer': 9,
  u'The Marshal': 1,
  u'William H. Rehnquist': 17},
 {nan: "0vary.twelve?But, it in...but but as you...categorization on the w- with your precise number...still...point.They're just...al-people...[Inaudible]Well, Mr.Mr. Ellis I I guess...commitment.and as...functioning of...[Inaudible]definition...[Inaudible]the heaviest penalty.situ-functioning.Right. Yes.yes.think it is.in in resp-appropriate punishment.and understanding. Yes.[Inaudible]States...rejected the notion.Am I right?consensus...last twenty years? I count two.consensus. Less than half.[Inaudible]I...consensus.[Inaud

time: 5.08 ms


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


time: 7.59 s


In [34]:
type(temp_dict2)

dict

time: 2.83 ms


In [23]:
#sort into lists
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []
list10 = []
list11 = []

for i in list_text4:
    list1.append(i[0])
    list2.append(i[1])
    list3.append(i[2])
    list4.append(i[3])
    list5.append(i[4])
    list6.append(i[5])
    list7.append(i[6])
    list8.append(i[7])
    list9.append(i[8])
    list10.append(i[9])
    list11.append(i[10])


time: 24.2 ms


In [24]:
#put lists in dataframe
df = pd.DataFrame()
df['link_trans'] = list1
df['docket'] = list2
df['respondent'] = list3
df['petitioner'] = list4
df['pet_lawyer'] = list5
df['resp_lawyer'] = list6
df['decided'] = list7
df['decision'] = list8
df['justice_counts'] = list9
df['speaker_verbage'] = list10
df['case_link'] = list11

time: 934 ms


In [25]:
#save data to csv file
df.to_csv('data_speaker_1990_2017.csv',encoding='utf-8')

time: 4.01 s


In [26]:
#get list of supreme court justices
html_justice = []
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")   
driver.get('https://www.oyez.org/justices')
sleep(1)
html_justice = driver.page_source
driver.close()

time: 3.76 s


In [27]:
soup_justice = BeautifulSoup(html_justice, 'lxml')

time: 213 ms


In [28]:
justices = []
for i in soup_justice.find_all('a', {'class':'ng-binding'}):         
    justices.append(i.text)

time: 57.2 ms


In [29]:
justices = justices[4:]

time: 2.25 ms


In [30]:
j_df = pd.DataFrame(justices)

time: 5.57 ms


In [32]:
j_df.to_csv('justice_list.csv')

time: 2.5 ms
